In [1]:
import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()

import pandas as pd
import numpy as np
from pyspark.sql.functions import col, expr
from pyspark.sql.functions import lit
from pyspark.sql.functions import *

print("Intalized")

Intalized


Create a spark data frame that contains your favorite programming languages.

* The name of the column should be language
* View the schema of the dataframe
* Output the shape of the dataframe
* Show the first 5 records in the dataframe


In [2]:
favorite_languages = ['python', 'c#', 'java', 'php', 'html']
rank = [1, 5, 3, 4, 2]
df = pd.DataFrame(favorite_languages, rank).reset_index()
df.columns=['rank', 'language']
df = spark.createDataFrame(df)
df.show()

+----+--------+
|rank|language|
+----+--------+
|   1|  python|
|   5|      c#|
|   3|    java|
|   4|     php|
|   2|    html|
+----+--------+



In [3]:
df.printSchema()

root
 |-- rank: long (nullable = true)
 |-- language: string (nullable = true)



In [4]:
df.describe().show()

+-------+------------------+--------+
|summary|              rank|language|
+-------+------------------+--------+
|  count|                 5|       5|
|   mean|               3.0|    null|
| stddev|1.5811388300841898|    null|
|    min|                 1|      c#|
|    max|                 5|  python|
+-------+------------------+--------+



Load the mpg dataset as a spark dataframe For each vehicle..

* Create 1 column of output that contains a message like the one below:
>The 1999 audi a4 has a 4 cylinder engine.
* Transform the trans column so that it only contains either manual or auto.

In [5]:
from pydataset import data

mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [6]:
mpg.select(
    concat(lit("The "), col("year"), lit(" "), 
    col("manufacturer"), lit(" "),
    col("model"), lit(" has a "),
    col("cyl"), lit(" cylinder engine"))
).show(truncate=False)

+-----------------------------------------------------------------------------+
|concat(The , year,  , manufacturer,  , model,  has a , cyl,  cylinder engine)|
+-----------------------------------------------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine                                     |
|The 1999 audi a4 has a 4 cylinder engine                                     |
|The 2008 audi a4 has a 4 cylinder engine                                     |
|The 2008 audi a4 has a 4 cylinder engine                                     |
|The 1999 audi a4 has a 6 cylinder engine                                     |
|The 1999 audi a4 has a 6 cylinder engine                                     |
|The 2008 audi a4 has a 6 cylinder engine                                     |
|The 1999 audi a4 quattro has a 4 cylinder engine                             |
|The 1999 audi a4 quattro has a 4 cylinder engine                             |
|The 2008 audi a4 quattro has a 4 cylind

In [11]:
mpg.select(
    concat(lit("The "), "year", lit(" "), 
    "manufacturer", lit(" "),
    "model", lit(" has a "),
    "cyl", lit(" cylinder engine"))
).show(truncate=False)

+-----------------------------------------------------------------------------+
|concat(The , year,  , manufacturer,  , model,  has a , cyl,  cylinder engine)|
+-----------------------------------------------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine                                     |
|The 1999 audi a4 has a 4 cylinder engine                                     |
|The 2008 audi a4 has a 4 cylinder engine                                     |
|The 2008 audi a4 has a 4 cylinder engine                                     |
|The 1999 audi a4 has a 6 cylinder engine                                     |
|The 1999 audi a4 has a 6 cylinder engine                                     |
|The 2008 audi a4 has a 6 cylinder engine                                     |
|The 1999 audi a4 quattro has a 4 cylinder engine                             |
|The 1999 audi a4 quattro has a 4 cylinder engine                             |
|The 2008 audi a4 quattro has a 4 cylind

In [9]:
mpg.select(regexp_replace("trans", r"\([^)]*\)", "").alias("transonly")).show()

+---------+
|transonly|
+---------+
|     auto|
|   manual|
|   manual|
|     auto|
|     auto|
|   manual|
|     auto|
|   manual|
|     auto|
|   manual|
|     auto|
|     auto|
|   manual|
|     auto|
|   manual|
|     auto|
|     auto|
|     auto|
|     auto|
|     auto|
+---------+
only showing top 20 rows



In [10]:
mpg.withColumn("trans", when(mpg.trans.startswith("a"), "auto").otherwise("manual")).show(6)

+------------+-----+-----+----+---+------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl| trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto|  f| 16| 26|  p|compact|
|        audi|   a4|  2.8|1999|  6|manual|  f| 18| 26|  p|compact|
+------------+-----+-----+----+---+------+---+---+---+---+-------+
only showing top 6 rows



Load the tips dataset as a spark dataframe.


* Use the seattle weather dataset referenced in the lesson to answer the questions below.

In [14]:
tips = spark.createDataFrame(data("tips"))
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



What percentage of observations are smokers?

In [15]:
tips.filter(tips.smoker == "Yes").count()/tips.count()

0.38114754098360654

Create a column that contains the tip percentage


In [16]:
tips.createOrReplaceTempView("tips")

In [17]:
spark.sql(
    """
SELECT tip, total_bill, round((tip/total_bill) * 100, 2) AS tip_percent
FROM tips
"""
).show(5)

+----+----------+-----------+
| tip|total_bill|tip_percent|
+----+----------+-----------+
|1.01|     16.99|       5.94|
|1.66|     10.34|      16.05|
| 3.5|     21.01|      16.66|
|3.31|     23.68|      13.98|
|3.61|     24.59|      14.68|
+----+----------+-----------+
only showing top 5 rows



In [18]:
tips.select(tips.tip/tips.total_bill).show()

+-------------------+
| (tip / total_bill)|
+-------------------+
|0.05944673337257211|
|0.16054158607350097|
|0.16658733936220846|
| 0.1397804054054054|
|0.14680764538430255|
|0.18623962040332148|
|0.22805017103762829|
|0.11607142857142858|
|0.13031914893617022|
| 0.2185385656292287|
| 0.1665043816942551|
|0.14180374361883155|
|0.10181582360570687|
|0.16277807921866522|
|0.20364126770060686|
|0.18164967562557924|
| 0.1616650532429816|
|0.22774708410067526|
|0.20624631703005306|
|0.16222760290556903|
+-------------------+
only showing top 20 rows



* Calculate the average tip percentage for each combination of sex and smoker.

In [19]:
tips.groupBy(tips.smoker, tips.sex).agg(round(avg(tips.tip/tips.total_bill * 100), 2)).show()

+------+------+-----------------------------------------+
|smoker|   sex|round(avg(((tip / total_bill) * 100)), 2)|
+------+------+-----------------------------------------+
|    No|Female|                                    15.69|
|    No|  Male|                                    16.07|
|   Yes|  Male|                                    15.28|
|   Yes|Female|                                    18.22|
+------+------+-----------------------------------------+



In [20]:
spark.sql(
"""
SELECT smoker, sex, avg(round((tip/total_bill) * 100, 2)) AS avg_tip_percent
FROM tips
group by smoker, sex
"""
).show(5)

+------+------+------------------+
|smoker|   sex|   avg_tip_percent|
+------+------+------------------+
|    No|Female|15.691111111111113|
|    No|  Male| 16.06659793814433|
|   Yes|  Male|15.276666666666666|
|   Yes|Female| 18.21454545454545|
+------+------+------------------+



Convert the temperatures to farenheight.
* What is the most frequent type of weather in January?
* What is the average high and low tempurature on sunny days in July in 2013 and 2014?
* What percentage of days were rainy in q3 of 2015?
* For each year, find what percentage of days it rained (had non-zero precipitation).

In [21]:
from vega_datasets import data
data.seattle_weather()

,date,precipitation,temp_max,temp_min,wind,weather
0,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,2012-01-02,10.9,10.6,2.8,4.5,rain
2,2012-01-03,0.8,11.7,7.2,2.3,rain
3,2012-01-04,20.3,12.2,5.6,4.7,rain
4,2012-01-05,1.3,8.9,2.8,6.1,rain
...,...,...,...,...,...,...
1456,2015-12-27,8.6,4.4,1.7,2.9,fog
1457,2015-12-28,1.5,5.0,1.7,1.3,fog
1458,2015-12-29,0.0,7.2,0.6,2.6,fog
1459,2015-12-30,0.0,5.6,-1.0,3.4,sun


In [22]:
temp = spark.createDataFrame(data("seattle_weather"))
temp.show(5)

+-------------------+-------------+--------+--------+----+-------+
|               date|precipitation|temp_max|temp_min|wind|weather|
+-------------------+-------------+--------+--------+----+-------+
|2012-01-01 00:00:00|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02 00:00:00|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03 00:00:00|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04 00:00:00|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05 00:00:00|          1.3|     8.9|     2.8| 6.1|   rain|
+-------------------+-------------+--------+--------+----+-------+
only showing top 5 rows



Which month has the most rain, on average?


In [52]:
(
    temp.withColumn("month", month("date"))
    .groupBy("month")
    .agg(avg("precipitation").alias("avg_rainfall"))
    .sort("avg_rainfall", ascending=False)
).show(1)

+-----+-----------------+
|month|     avg_rainfall|
+-----+-----------------+
|   11|5.354166666666667|
+-----+-----------------+
only showing top 1 row



Which year was the windiest?

In [55]:
(
    temp.withColumn("year", year("date"))
    .groupBy("year")
    .agg(avg("wind").alias("year_wind"))
    .sort("year_wind", ascending=False)
).show(1)

+----+-----------------+
|year|        year_wind|
+----+-----------------+
|2012|3.400819672131147|
+----+-----------------+
only showing top 1 row



In [56]:
# Using sum
(
    temp.withColumn("year", year("date"))
    .groupBy("year")
    .agg(sum("wind").alias("year_wind"))
    .sort("year_wind", ascending=False)
).show(1)

+----+------------------+
|year|         year_wind|
+----+------------------+
|2012|1244.6999999999998|
+----+------------------+
only showing top 1 row

